# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

O problema do bloco deslizante apresenta uma matriz 3x3 elementos que podem ser rearranjados em qualquer posição, ou seja, o número de estados possíveis consiste no número de permutações possíveis dessa matriz: `(3x3)! = 9! = 362880`. O objetivo desse problema é arranjar os elementos de forma que eles estejam em ordem crescente, com o espaço vázio na última posição.

Para representar os estados, serão utilizadas strings que codificam a matriz em sua ordem de valores. O valor 0 representará o espaço vazio. Abaixo está um exemplo dessa codificação:

```
[[1, 0, 3],
 [4, 6, 5],  = "103465782"
 [7, 8, 2]]
```

Essa codificação permite que os estados sejam armazenados em um `set` de estados já visitados, fazendo com que não sejam visitados estados repetidos. Matrizes são um tipo de dado mutável, portanto não é possível armazená-las em `sets`.

O valor de retorno das funções que resolvem o problema é uma lista posições do espaço vazio. Ao saber a posição original da matriz, e qual será a próxima posição do espaço vazio, podemos obter iterativamente todo o caminho traçado até chegar na solução. O tamanho do vetor representa o número de movimentos necessários para resolver o problema, com a exceção do primeiro elemento, que representa a posição inicial do espaço vazio.

# Resolver o quebra-cabeças usando Buscas

In [2]:
import itertools

import numpy as np


possible_moves = [
    (0, -1),  (1, 0), 
    (-1, 0),  (0, 1)
]


class State(object):
    def __init__(self, matrix):
        # Knowledge
        self.matrix = matrix

        # Representation
        self.representation = self.encode(matrix)
    
    def encode(self, matrix):
        # Flatten the matrix
        array = list(itertools.chain.from_iterable(matrix))

        # Convert to string
        return "".join(str(elem) for elem in array)
    
    def get_empty_position(self):
        # Find empty position (represented by 0)
        line, column = np.where(self.matrix == 0)
        line = line[0]
        column = column[0]

        empty_position = (line, column)

        return empty_position

    def get_moves(self):
        empty_position = self.get_empty_position()
        line, column = empty_position

        # Get positions that can move
        moves = list()

        for horizontal, vertical in possible_moves:
            position = ((line + vertical), (column + horizontal))

            # Verify if all index are smaller than 3
            if all(-1 < pos < 3 for pos in position):
                moves.append((position))
        
        return moves, empty_position
    
    def __del__(self):
        # Free the memory allocated by the matrix
        del self.matrix


def swap_matrix(matrix, position1, position2):
    new_matrix = matrix.copy()

    new_matrix[position1], new_matrix[position2] = new_matrix[position2], new_matrix[position1]

    return new_matrix


def print_moves(solving_moves, matrix_input):
    matrix = matrix_input.copy()
    
    for index in range(len(solving_moves) - 1):
        empty_pos = solving_moves[index]
        new_pos = solving_moves[index + 1]

        print(matrix, '\n')
        matrix[empty_pos], matrix[new_pos] = matrix[new_pos], matrix[empty_pos]
    
    print(matrix)

## Busca em Profundidade

No algoritmo DFS, os estados são visitados sequencialmente até atingir um ponto onde não há mais estados que possam ser visitados. Quando isso ocorre, o algoritmo retorna até um ponto anterior onde é possível verificar mais estados. Esse algoritmo foi implementado de forma recursiva, sendo o pior caso da pilha de chamadas igual a 362880, que é o número máximo de permutações da matriz.

In [3]:
# Increase the recursion limit
import resource
import sys

resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

In [4]:
visited_states = set()
solution = "123456780"


def solve_sliding_puzzle_dfs(state, moves):
    # Recursion base
    if state.representation == solution:
        # The solution was found
        solving_moves = moves.copy()
        return solving_moves

    visited_states.add(state.representation)

    # Recursion step
    current_moves, empty_pos = state.get_moves()

    for move_pos in current_moves:
        # Swap empty position to move position
        new_matrix = swap_matrix(state.matrix, move_pos, empty_pos)

        new_state = State(new_matrix)

        # Verify if state has already been visited
        if new_state.representation in visited_states:
            # Free up state memory
            del new_state
            continue

        # Add move position to moves
        moves.append(move_pos)

        solving_moves = solve_sliding_puzzle_dfs(new_state, moves)

        if solving_moves is not None:
            return solving_moves
        
        # Free up state memory
        del new_state

        moves.pop()

    return solving_moves


matrix = np.array(
    [[1, 2, 3],
     [4, 5, 6],  
     [7, 0, 8]]
)

initial_state = State(matrix.copy())
initial_position = [initial_state.get_empty_position()]

solving_moves = solve_sliding_puzzle_dfs(initial_state, initial_position)

print(f"Number of moves to solve the problem: {len(solving_moves) - 1}")

Number of moves to solve the problem: 41


In [5]:
print_moves(solving_moves, matrix)

[[1 2 3]
 [4 5 6]
 [7 0 8]] 

[[1 2 3]
 [4 0 6]
 [7 5 8]] 

[[1 0 3]
 [4 2 6]
 [7 5 8]] 

[[1 3 0]
 [4 2 6]
 [7 5 8]] 

[[1 3 6]
 [4 2 0]
 [7 5 8]] 

[[1 3 6]
 [4 0 2]
 [7 5 8]] 

[[1 0 6]
 [4 3 2]
 [7 5 8]] 

[[1 6 0]
 [4 3 2]
 [7 5 8]] 

[[1 6 2]
 [4 3 0]
 [7 5 8]] 

[[1 6 2]
 [4 0 3]
 [7 5 8]] 

[[1 0 2]
 [4 6 3]
 [7 5 8]] 

[[1 2 0]
 [4 6 3]
 [7 5 8]] 

[[1 2 3]
 [4 6 0]
 [7 5 8]] 

[[1 2 3]
 [4 6 8]
 [7 5 0]] 

[[1 2 3]
 [4 6 8]
 [7 0 5]] 

[[1 2 3]
 [4 0 8]
 [7 6 5]] 

[[1 0 3]
 [4 2 8]
 [7 6 5]] 

[[1 3 0]
 [4 2 8]
 [7 6 5]] 

[[1 3 8]
 [4 2 0]
 [7 6 5]] 

[[1 3 8]
 [4 0 2]
 [7 6 5]] 

[[1 0 8]
 [4 3 2]
 [7 6 5]] 

[[1 8 0]
 [4 3 2]
 [7 6 5]] 

[[1 8 2]
 [4 3 0]
 [7 6 5]] 

[[1 8 2]
 [4 0 3]
 [7 6 5]] 

[[1 0 2]
 [4 8 3]
 [7 6 5]] 

[[1 2 0]
 [4 8 3]
 [7 6 5]] 

[[1 2 3]
 [4 8 0]
 [7 6 5]] 

[[1 2 3]
 [4 8 5]
 [7 6 0]] 

[[1 2 3]
 [4 8 5]
 [7 0 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 0 3]
 [4 2 5]
 [7 8 6]] 

[[1 3 0]
 [4 2 5]
 [7 8 6]] 

[[1 3 5]
 [4 2 0]
 [7 8 6]] 

[[1 3 5]
 

## Busca em Largura

O algoritmo BFS visita todos os estados vizinhos antes de avançar para o próximo estado. Isso faz com que esse algoritmo encontre soluções melhores que DFS. No entanto, isso também ocasiona uma complexidade muito maior na sua busca, tornando inviável sua execução para certos valores de entrada. Esse algoritmo foi implementado de forma iterativa utilizando uma fila que armazena os estados e os movimentos utilizados até o momento.

In [6]:
from collections import deque

visited_states = set()
solution = "123456780"
queue = deque()


def solve_sliding_puzzle_bfs(state, moves):
    visited_states.add(state.representation)

    # Verify if initial state is the solution
    if state.representation == solution:
        # The solution was found
        return moves

    queue.append((state, moves))

    while len(queue) != 0:
        current_state, state_moves = queue.popleft()

        current_moves, empty_pos = current_state.get_moves()

        for move_pos in current_moves:
            # Swap empty position to move position
            new_matrix = swap_matrix(state.matrix, move_pos, empty_pos)

            new_state = State(new_matrix)

            # Verify if state has already been visited
            if new_state.representation in visited_states:
                # Free up state memory
                del new_state
                continue

            # Add move position to moves
            new_moves = state_moves.copy()

            new_moves.append(move_pos)

            queue.append((new_state, new_moves))

            if new_state.representation == solution:
                # The solution was found
                return new_moves
            
        # Free up memory
        del current_state
        del state_moves

    return None

matrix = np.array(
    [[1, 2, 3],
     [4, 5, 6],  
     [7, 0, 8]]
)

initial_state = State(matrix.copy())
initial_position = [initial_state.get_empty_position()]

solving_moves = solve_sliding_puzzle_bfs(initial_state, initial_position)

print(f"Number of moves to solve the problem: {len(solving_moves) - 1}")

Number of moves to solve the problem: 1


In [7]:
print_moves(solving_moves, matrix)

[[1 2 3]
 [4 5 6]
 [7 0 8]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória

O consumo de memória se mostrou mais grave no algoritmo de busca em largura do que na busca em profundidade, havendo casos em que a busca em largura atingiu o limite de RAM disponibilizado pelo Google Colab antes de concluir sua execução. Já a busca em profunidade encontrou uma solução em todos os casos testados.

No DFS, apenas o estado atual e todos os estados visitados até o momento (codificados de forma a reduzir o consumo de memória) devem ser armazenados. Já o BFS, além de armazenar as mesmas informações do DFS, utiliza também uma fila que guarda os estados que devem ser testados em seguida, e uma lista das posições percorridas para chegar em cada um desses estado. Esse uso crítico de memória utilizada pelo BFS pode ser explicado também por limitações na implementação do algoritmo e representação dos dados. 

2.   Processamento

O processamento em ambos os algoritmos não se mostrou como um grande gargalo em sua execução, quando comparado ao seu excessivo uso de memória. Ainda assim, o BFS apresentou um maior tempo de execução para a maioria dos casos analisados, visto que mais estados serão testados antes de encontrar a solução. O algoritmo DFS foi implementado de forma recursiva, exigindo que o limite de recursão do sistema fosse aumentado antes de sua execução. Ademais, ambos os algoritmos trabalham com manipulação de matrizes, que podem demandar um maior poder de processamento.
